# Assignment 1

In [ ]:
import cv2
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
from IPython.display import Markdown
from scipy import stats

### Parameters

In [ ]:
im_count = 8   # Image Count
bins = 256     # Numbers of Bins
r_min = np.uint8(0)      # Minimum Intensity - Dynamic Range
r_max = np.uint8(255)    # Maximum Intensity - Dynamic Range
colorSpace = 'CIELAB' # Color space to perform equalization in: CIELAB or YCrCb.
saveImages = True

# Contrast Limited Adaptive Histogram Equalization Params
win_size = 64 # Width/height of sliding window
max_slope = 8 # Max slope of cdf, used for calculating clip value
clip_tolerance = 1.0 # Tolerance between max histogram bin value vs clip value

# matplotlib Params
plotCharts = True
tick_step = 5  # X-Axis Steps on Charts
dpi = mpl.rcParams['figure.dpi'] # DPI used for matplotlib figure drawing

### Load and display images

In [ ]:
im = [0] * im_count

for i in range(1, im_count + 1):
  im[i - 1] = cv2.imread('sample' + '%.2d'%i + '.jpg', cv2.IMREAD_UNCHANGED)
  if im[i - 1] is None:
    im[i - 1] = cv2.imread('sample' + '%.2d'%i + '.jpeg', cv2.IMREAD_UNCHANGED)
  im[i - 1] = cv2.cvtColor(im[i - 1], cv2.COLOR_BGR2RGB)

In [ ]:
for i in im:
  display(Image.fromarray(i))

### Convert BGR to CIELAB/YCrCb and display lightness/luma channel

In [ ]:
def displayIm(im, filename = "", save = False):
  imPIL = Image.fromarray(im)
  display(imPIL)
  if save:
    imPIL.save(filename)

imLuma = [0] * im_count

for i in range(len(im)):
  if colorSpace == 'CIELAB':
    im[i] = cv2.cvtColor(im[i], cv2.COLOR_RGB2LAB)
  elif colorSpace == 'YCrCb':
    im[i] = cv2.cvtColor(im[i], cv2.COLOR_RGB2YCrCb)
  else:
    print("Invalid colorSpace")
    break
  imLuma[i] = im[i][:,:,0]

In [ ]:
for i in range(len(imLuma)):
  if colorSpace == 'CIELAB':
    displayIm(imLuma[i], "[LIGHTNESS]Sample" + "%.2d"%(i+1) + ".png", saveImages)
  elif colorSpace == 'YCrCb':
    displayIm(imLuma[i], "[LUMA]Sample" + "%.2d"%(i+1) + ".png", saveImages)

### Plot frequency and probability histograms

In [ ]:
def histogram(im, bins):
  freqHist = [0] * bins
  probHist = [0] * bins
  kdeBin = [0] * bins
  imData = im.reshape(im.shape[0] * im.shape[1])
  pixelCount = len(imData)
  binSize = 256/bins
  for pixelValue in imData:
    freqHist[math.floor(pixelValue / binSize)] += 1
  for i in range(bins):
    probHist[i] = freqHist[i] / pixelCount

  intensities = list(range(256))
  kde = stats.gaussian_kde(imData)
  kde = kde(intensities)
  for i in range(256):
    kdeBin[math.floor(i / binSize)] += kde[i]
  return freqHist, probHist, kdeBin

def plotHist(freqHist, probHist, kde, title, tick_step):
  bins = list(range(len(freqHist)))
  xTicks = list(range(0, len(freqHist), tick_step))
  xlabel = r'Intensity, $r_k$'
  ylabelFreq = r'Frequency, $n_k$'
  ylabelProb = r'Probability, $p_r(r_k) = \frac{n_k}{MN}$'
  fontsize = 22
  intensities = list(range(len(freqHist)))

  figure, axes = plt.subplots(2, 1, figsize = (1280./dpi,640./dpi), tight_layout = True)

  axes[0].bar(bins, freqHist, color = 'xkcd:light blue', width = 0.5)
  axes[1].bar(bins, probHist, color = 'xkcd:light blue', width = 0.5)

  axesFreqKde = axes[0].inset_axes([0,0,1,1])
  axesFreqKde.plot(intensities, kde, color ='xkcd:orange')
  axesFreqKde.patch.set_visible(False)
  axesFreqKde.axis('off')
  axesFreqKde.set_ylim(axes[1].get_ylim())
  axesFreqKde.set_xlim(-0.5, len(freqHist) - 0.5)

  axesProbKde = axes[1].inset_axes([0,0,1,1])
  axesProbKde.plot(intensities, kde, color ='xkcd:orange')
  axesProbKde.patch.set_visible(False)
  axesProbKde.axis('off')
  axesProbKde.set_ylim(axes[1].get_ylim())
  axesProbKde.set_xlim(-0.5, len(freqHist) - 0.5)

  for ax in axes:
    ax.set_title(title, fontsize = fontsize, fontweight = 'bold')
    ax.set_ylim(0)
    ax.set_xlim(-0.5, len(freqHist) - 0.5)
    ax.set_xlabel(xlabel, fontsize = fontsize)
    ax.set_xticks(xTicks)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
  axes[0].set_ylabel(ylabelFreq, fontsize = fontsize)
  axes[1].set_ylabel(ylabelProb, fontsize = fontsize)
  figure.savefig("[HIST]" + title + ".png")
  plt.show()

In [ ]:
freqHist = [0] * len(im)
probHist = [0] * len(im)
kde = [0] * len(im)

for i in range(len(im)):
  freqHist[i], probHist[i], kde[i] = histogram(imLuma[i], bins)
  if plotCharts: plotHist(freqHist[i], probHist[i], kde[i], "Sample" + "%.2d"%(i+1), tick_step)

### Plot cumulative distribution function *w.r.t* probability of occurence pixel value *i*

In [ ]:
def cdf(probHist):
  cdf = []
  cumul = 0
  for i in probHist:
    cumul += i
    cdf.append(cumul)
  return cdf

def plotCdf(cdf, probHist, kde, title, tick_step, **kwargs):
  intensities = list(range(len(cdf)))
  xTicks = list(range(0, len(cdf), tick_step))
  xlabel = r'Intensity, $r_k$'
  ylabelCdf = r'$cdf_r(r_k)=\sum_{j=0}^k p_r(r_j)$'
  ylabelProb = r'Probability, $p_r(r_k) = \frac{n_k}{MN}$'
  fontsize = 22
  if 'colorMap' in kwargs.keys():
    if kwargs['colorMap'] == 'original':
      color = list(str(i / 256) for i in range(0, 256, int(256 / len(cdf))))
    else:
      color = [str(transform / 256) for transform in kwargs['colorMap']]
    edgecolor = 'xkcd:light grey'
  else:
    color = 'xkcd:light blue'
    edgecolor = None

  fig, ax = plt.subplots(figsize = (1280./dpi,640./dpi), tight_layout = True)
  fig.suptitle(title, fontsize = fontsize, fontweight = 'bold')
  ax2 = ax.twinx()
  ax2.bar(intensities, probHist, color = color, edgecolor = edgecolor, width = 0.7)
  ax2.plot(intensities, kde, color ='xkcd:orange')
  ax2.set_ylabel(ylabelProb, fontsize = fontsize)
  ax2.spines['top'].set_visible(False)
  ax.plot(intensities, cdf, color ='xkcd:light red')
  ax.set_ylim(0)
  ax.set_xlim(-0.5, len(cdf) - 0.5)
  ax.set_ylabel(ylabelCdf, fontsize = fontsize)
  ax.set_xlabel(xlabel, fontsize = fontsize)
  ax.set_xticks(xTicks)
  ax.spines['top'].set_visible(False)
  ax.set_zorder(1)
  ax.patch.set_visible(False)
  fig.savefig("[CDF]" + title + ".png")
  plt.show()

In [ ]:
probCdf = [0] * len(im)

for i in range(len(im)):
  probCdf[i] = cdf(probHist[i])
  if plotCharts:
    plotCdf(probCdf[i], probHist[i], kde[i], "Sample" + "%.2d"%(i+1), tick_step)

### Perform histogram equalization based on $cdf_r(r_k)$

In [ ]:
def histEq(im, cdf, bins, r_min, r_max):
  r = r_max - r_min
  binSize = 256/bins
  outIm = np.zeros((im.shape[0], im.shape[1]), np.uint8)
  transform = [0] * len(cdf)
  for i in range(len(cdf)):
    transform[i] = int(round((cdf[i] * r) + r_min))
  for i in range(im.shape[0]):
    for j in range(im.shape[1]):
      outIm[i][j] = transform[math.floor(im[i][j] / binSize)]
  return outIm, transform

In [ ]:
eqImLuma = [0] * len(im)
eqFreqHist = [0] * len(im)
eqProbHist = [0] * len(im)
eqKde = [0] * len(im)
eqCdf = [0] * len(im)
transformMap = [0] * len(im)

for i in range(len(im)):
  eqImLuma[i], transformMap[i] = histEq(imLuma[i], probCdf[i], bins, r_min, r_max)
  if plotCharts:
    eqFreqHist[i], eqProbHist[i], eqKde[i] = histogram(eqImLuma[i], bins)
    eqCdf[i] = cdf(eqProbHist[i])
    plotCdf(probCdf[i], probHist[i], kde[i], "[COLOR TRANSFORM MAPPING]Sample" + "%.2d"%(i+1), tick_step, colorMap = transformMap[i]) # Colors histogram bars with mapped transform intensities
    plotCdf(probCdf[i], probHist[i], kde[i], "[COLOR]Sample" + "%.2d"%(i+1), tick_step, colorMap = 'original')  # Colors histogram bars with respective intensities
    plotCdf(eqCdf[i], eqProbHist[i], eqKde[i], "[COLOR][HE]Sample" + "%.2d"%(i+1), tick_step, colorMap = 'original')
    plotCdf(eqCdf[i], eqProbHist[i], eqKde[i], "[HE]Sample" + "%.2d"%(i+1), tick_step)

### Merge equalized channel back into image

In [ ]:
eqIm = [0] * len(im)

for i in range(len(im)):
  display(Markdown('### Sample %.2d'%(i+1)))
  displayIm(imLuma[i])
  if colorSpace == 'CIELAB':
    displayIm(eqImLuma[i], "[HE][LIGHTNESS]Sample" + "%.2d"%(i+1) + ".png", saveImages)
    displayIm(cv2.cvtColor(im[i], cv2.COLOR_LAB2RGB))
    eqIm[i] = cv2.merge((eqImLuma[i], im[i][:,:,1], im[i][:,:,2]))
    eqIm[i] = cv2.cvtColor(eqIm[i], cv2.COLOR_LAB2RGB)
    displayIm(eqIm[i], "[HE][CIELAB]Sample" + "%.2d"%(i+1) + ".png", saveImages)
  elif colorSpace == 'YCrCb':
    displayIm(eqImLuma[i], "[HE][LUMA]Sample" + "%.2d"%(i+1) + ".png", saveImages)
    displayIm(cv2.cvtColor(im[i], cv2.COLOR_YCrCb2RGB))
    eqIm[i] = cv2.merge((eqImLuma[i], im[i][:,:,1], im[i][:,:,2]))
    eqIm[i] = cv2.cvtColor(eqIm[i], cv2.COLOR_YCrCb2RGB)
    displayIm(eqIm[i], "[HE][YCRCB]Sample" + "%.2d"%(i+1) + ".png", saveImages)
  else:
    print("Invalid colorSpace")
    break

### Perform Linear Contrast Stretching

In [ ]:
def lcs(im):
  r_min_lcs = 255
  r_max_lcs = 0
  r_lcs = 0
  outIm = np.zeros((im.shape[0], im.shape[1]), np.uint8)
  for i in range(im.shape[0]):
    for j in range(im.shape[1]):
      r_min_lcs = im[i][j] if im[i][j] < r_min_lcs else r_min_lcs
      r_max_lcs = im[i][j] if im[i][j] > r_max_lcs else r_max_lcs
  r_lcs = r_max_lcs - r_min_lcs
  for i in range(im.shape[0]):
    for j in range(im.shape[1]):
      outIm[i][j] = round(255 * (im[i][j] - r_min_lcs) / r_lcs)
  return outIm

In [ ]:
imLcs = [0] * len(im)
lcsFreqHist = [0] * len(im)
lcsProbHist = [0] * len(im)
lcsKde = [0] * len(im)
lcsProbCdf = [0] * len(im)

for i in range(len(imLuma)):
  imLcs[i] = lcs(imLuma[i])
  displayIm(imLcs[i], "[LCS]Sample" + "%.2d"%(i+1) + ".png", saveImages)
  if plotCharts:
    lcsFreqHist[i], lcsProbHist[i], lcsKde[i] = histogram(imLcs[i], 256)
    lcsProbCdf[i] = cdf(lcsProbHist[i])
    plotCdf(lcsProbCdf[i], lcsProbHist[i], lcsKde[i], "[LCS]Sample" + "%.2d"%(i+1), tick_step)

### Perform Sliding Window Contrast Limited Adaptive Histogram Equalization

In [ ]:
def clahe_transform(imOut, imIn, im_x, im_y, freqHist, pixelCount, max_slope, clip_tolerance):
  freqHistClip = freqHist.astype(np.float64)
  clipValue = pixelCount/256 * max_slope

  while (np.amax(freqHistClip) - clipValue) > clip_tolerance:
    clipSum = 0
    for i in range(256):
      if freqHistClip[i] > clipValue:
        clipSum += freqHistClip[i] - clipValue
        freqHistClip[i] = clipValue
    freqHistClip = freqHistClip + (clipSum / 256)

  cHist = 0
  iPixel = imIn[im_y][im_x]
  if (iPixel <= 128):
    for i in range(iPixel + 1):
      cHist += freqHistClip[i]
  else:
    for i in range(iPixel + 1, 256):
      cHist += freqHistClip[i]
    cHist = pixelCount - cHist
  imOut[im_y][im_x] = round((cHist/pixelCount) * 255)

def clahe(im, win_size, max_slope, clip_tolerance):
  pixelCount = win_size**2
  shift = int(math.log2((win_size**2)/256))

  imPadded = np.pad(im, int(win_size/2), mode = 'symmetric')
  imNew = np.zeros([im.shape[0],im.shape[1]], np.uint8)
  freqHist = np.zeros(256, np.uint32)

  for im_y in range(im.shape[0]):
    if im_y == 0:
      # Initialize freqHist with histogram at top-left pixel original image
      for windowRow in imPadded[1:win_size+1, 1:win_size+1]:
        for pixelValue in windowRow:
          freqHist[pixelValue] += 1
    else:
      # Subtract trailing row
      for pixelValue in imPadded[im_y, 1:win_size+1]:
        freqHist[pixelValue] -= 1
      # Add leading row
      for pixelValue in imPadded[win_size+im_y, 1:win_size+1]:
        freqHist[pixelValue] += 1

    # Perform HE mapping
    clahe_transform(imNew, im, 0, im_y, freqHist, pixelCount, max_slope, clip_tolerance)

    freqHistX = np.copy(freqHist)
    for im_x in range(1, im.shape[1]):
      # Subtract trailing column
      for pixelValue in imPadded[im_y+1:im_y+win_size+1, im_x]:
        freqHistX[pixelValue] -= 1
      # Add leading column
      for pixelValue in imPadded[im_y+1:im_y+win_size+1, im_x+win_size]:
        freqHistX[pixelValue] += 1
      # Perform HE mapping
      clahe_transform(imNew, im, im_x, im_y, freqHistX, pixelCount, max_slope, clip_tolerance)
  return imNew

In [ ]:
imCLAHE = [0] * len(im)
claheFreqHist = [0] * len(im)
claheProbHist = [0] * len(im)
claheKde = [0] * len(im)
claheProbCdf = [0] * len(im)

for i in range(len(imLuma)):
  imCLAHE[i] = clahe(imLuma[i], win_size, max_slope, clip_tolerance)
  displayIm(imCLAHE[i], "[CLAHE]Sample" + "%.2d"%(i+1) + ".png", saveImages)
  if plotCharts:
    claheFreqHist[i], claheProbHist[i], claheKde[i] = histogram(imCLAHE[i], 256)
    claheProbCdf[i] = cdf(claheProbHist[i])
    plotCdf(claheProbCdf[i], claheProbHist[i], claheKde[i], "[CLAHE]Sample" + "%.2d"%(i+1), tick_step)